# **Day16**: *The Floor Will Be Lava*

## Part 1

With the beam of light completely focused somewhere, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.

Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.

Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing empty space (.), mirrors (/ and \), and splitters (| and -).

The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into heat to melt the rock in the cavern.

You note the layout of the contraption (your puzzle input). For example:

```
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
```

The beam enters in the top-left corner from the left and heading to the right. Then, its behavior depends on what it encounters as it moves:

- If the beam encounters empty space (.), it continues in the same direction.
- If the beam encounters a mirror (/ or \\), the beam is reflected 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a / mirror would continue upward in the mirror's column, while a rightward-moving beam that encounters a \ mirror would continue downward from the mirror's column.
- If the beam encounters the pointy end of a splitter (| or -), the beam passes through the splitter as if the splitter were empty space. For instance, a rightward-moving beam that encounters a - splitter would continue in the same direction.
- If the beam encounters the flat side of a splitter (| or -), the beam is split into two beams going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a | splitter would split into two beams: one that continues upward from the splitter's column and one that continues downward from the splitter's column.

Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is energized if that tile has at least one beam pass through it, reflect in it, or split in it.

In the above example, here is how the beam of light bounces around the contraption:

```
>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..
```

Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is energized (#) or not (.):

```
######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
```

Ultimately, in this example, 46 tiles become energized.

The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, how many tiles end up being energized?

### Solution

In [1]:
direction_map = {
    '/': ['RU', 'LD', 'UR', 'DL'],
    '\\': ['RD', 'LU', 'UL', 'DR'],
    '-': ['RR', 'LL', 'ULR', 'DLR'],
    '|': ['RDU', 'LUD', 'UU', 'DD'],
    '.': ['RR', 'LL', 'UU', 'DD']
}
move_map = {'R': (0, 1), 'L': (0, -1), 'U': (-1, 0), 'D': (1, 0)}


In [2]:
def take_step(position, direction, lines, visited:set, grid=None):
    # grid for debugging
    # grid = [[0 for _ in lines[0]] for _ in lines] if grid is None else grid
    # grid[position[0]][position[1]] = 1

    # check if we already looked at this beam
    visited.add((position, direction))
    
    # current position and symbol
    row, col = position
    symbol = lines[row][col]

    # next position and direction
    for option in direction_map[symbol]:
        if option[0] == direction:
            for new_direction in option[1:]:
                new_position = (row + move_map[new_direction][0], col + move_map[new_direction][1])
                if 0 <= new_position[0] < len(lines) and 0 <= new_position[1] < len(lines[0]):
                    if (new_position, new_direction) not in visited:
                        visited = take_step(new_position, new_direction, lines, visited, grid)
            
    return visited 

    

In [3]:
def get_number_of_energized_tiles(input_string):
    lines = input_string.splitlines()
    start_direction = 'R'
    start_position = (0, 0)

    visited = take_step(start_position, start_direction, lines, set())
    return len(set(v[0] for v in visited))
    


### Example

In [4]:
input_string = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|...."""

get_number_of_energized_tiles(input_string)


46

### Submission

In [5]:
import sys
sys.setrecursionlimit(10000)

with open('input/day16.txt', 'r') as file:
    input_string = file.read()

get_number_of_energized_tiles(input_string)


7392

## Part 2

As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from any edge tile and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)

So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that energizes as many tiles as possible.

In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:

```
.|<2<\....
|v-v\^....
.v.v.|->>>
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
<-2-/vv|..
.|<<<2-|.\
.v//.|.v..
```

Using this configuration, 51 tiles are energized:

```
.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..
```

Find the initial beam configuration that energizes the largest number of tiles; how many tiles are energized in that configuration?

### Solution

In [6]:
def get_number_of_energized_tiles(input_string):
    """
    I just brute forced it ¯\_(ツ)_/¯
    still acceptable runtime: 5.7 seconds
    """
    lines = input_string.splitlines()
    tiles = []

    # left side
    for i in range(len(lines)):
        start_direction = 'R'
        start_position = (i, 0)
        visited = take_step(start_position, start_direction, lines, set())
        tiles.append(len(set(v[0] for v in visited)))
    
    # right side
    for i in range(len(lines)):
        start_direction = 'L'
        start_position = (i, len(lines[0])-1)
        visited = take_step(start_position, start_direction, lines, set())
        tiles.append(len(set(v[0] for v in visited)))
    
    # top side
    for i in range(len(lines[0])):
        start_direction = 'D'
        start_position = (0, i)
        visited = take_step(start_position, start_direction, lines, set())
        tiles.append(len(set(v[0] for v in visited)))

    # bottom side
    for i in range(len(lines[0])):
        start_direction = 'D'
        start_position = (len(lines)-1, i)
        visited = take_step(start_position, start_direction, lines, set())
        tiles.append(len(set(v[0] for v in visited)))

    return max(tiles)


### Example

In [7]:
input_string = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|...."""

get_number_of_energized_tiles(input_string)

51

### Submission

In [8]:
import sys
sys.setrecursionlimit(10000)

with open('input/day16.txt', 'r') as file:
    input_string = file.read()

get_number_of_energized_tiles(input_string)


7665